In [1]:
### Binary classification -> Neural networks.
###classifying circles

In [6]:
##dependencies
from sklearn.datasets import make_circles
import pandas as pd


In [7]:
## make   1000 samples 
n_samples = 1000
x, y = make_circles(
    n_samples,
    noise=0.02,
    random_state=30
)

In [11]:
## visualizing my data
##dataframe
circles = pd.DataFrame({
    'X1': x[:, 0], 
    'X2': x[:, 1],
    'Labels': y
})

circles.head(10)

,X1,X2,Labels
0,0.471109,-0.688698,1
1,0.470924,-0.657534,1
2,0.757632,0.133427,1
3,0.441729,0.914881,0
4,0.064846,-0.799789,1
5,0.913613,-0.459859,0
6,0.182394,0.992764,0
7,-0.799707,-0.551241,0
8,0.488100,-0.869731,0
9,-0.357087,0.743005,1
